In [15]:
# Import packages.
import string
from io import BytesIO
from tensorflow.python.lib.io import file_io
import msgpack
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2VecKeyedVectors

In [16]:
raw_embed = Word2VecKeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
vocab = {}
vectors = [
    np.zeros((300, ), dtype='float32'),
]
for word in raw_embed.vocab:  # TODO IMPROVEMENT Is checking for non-ascii like this necessary?
    allowed = True
    for letter in word:
        if letter not in string.ascii_lowercase:  # TODO IMPROVEMENT Are valid ascii words that are not lowercase being rejected?
            allowed = False  # I don't see any step to force the words to lowercase before this check.
            break
    if allowed:
        vocab[word] = len(vectors)
        vectors.append(raw_embed.vectors[raw_embed.vocab[word].index])
# TODO IMPROVEMENT Avoid the problems of serializing numpy arrays by separating
# the vocab from vectors.
embed = {
    'vocab': vocab,
    'vectors': np.array(vectors),
    }
# with open(cache_filename, 'wb') as f_out:
#     pickle.dump(embed, f_out, pickle.HIGHEST_PROTOCOL)

In [17]:
GoogleNews_word2id = embed['vocab']

In [18]:
word2id_df = pd.DataFrame.from_dict(GoogleNews_word2id, orient='index', columns=['id'])
print("ln(word2id_df):", len(word2id_df))
word2id_df.head()

ln(word2id_df): 155060


,id
in,1
for,2
that,3
is,4
on,5


In [19]:
id2word_df = word2id_df.reset_index()
id2word_df.head()

,index,id
0,in,1
1,for,2
2,that,3
3,is,4
4,on,5


In [20]:
id2word_df = id2word_df.set_index('id')
id2word_df.head()

,index
id,
1,in
2,for
3,that
4,is
5,on


In [41]:
# Create dictionary of just IDs to words.
id2word_dict = id2word_df.to_dict()
id2word_dict = id2word_dict['index']

In [46]:
# Switch keys/values and store word2id dictionary.
# Needed to encode examples.
word2id_dict = {y: x for x, y in id2word_dict.items()}

In [47]:
with open('GoogleNews_word2id.bin', 'wb') as f:
    msgpack.pack(word2id_dict, f)

In [43]:
with open('GoogleNews_id2word.bin', 'wb') as f:
    msgpack.pack(id2word_dict, f)

In [33]:
GoogleNews_vectors = embed['vectors']
np.save('GoogleNews_Embeddings', GoogleNews_vectors)

In [34]:
# Decodes encoded text using the id2word dict.
def indexes_to_text(indexes):
    found_indexes_list = []
    not_found_indexes_list = []

    for index in indexes:
        if id2word.get(index) is not None:
            found_indexes_list.append(id2word_dict.get(index))
        else:
            not_found_indexes_list.append(index)

    print('Indexes not found:', not_found_indexes_list)

    return ' '.join(found_indexes_list)

In [48]:
# Read in features and labels.
f = BytesIO(file_io.read_file_to_string('wikimedia-personal-attacks-min-6-votes-GN-Encoded-data.bin', binary_mode=True))
data = msgpack.unpack(f, raw=False)

In [49]:
# For keeping number of words in longest document.
max_words = 0

# Create lists to store docs and labels.
docs = []
labels = []

# Iterate over data to build lists of docs and labels.
num_docs = len(data)
for i in range(num_docs):
#     sys.stdout.write("processing record %i of %i       \r" % (i + 1, num_docs))
#     sys.stdout.flush()

    # Get index of document.
    doc = data[i]['idx']

    # Retrieve document from saved data and cast to array.
    doc = [item for sublist in doc for item in sublist]

    # Add document to docs array.
    docs.append(doc)

    # Add label to label array at same index.
    labels.append(data[i]['label'])

    # Track maximum number of words in document.
    if len(doc) > max_words:
        max_words = len(doc)

In [51]:
len(docs)

115841

In [ ]:
# Also create plain text version of GN vectors for retrofitting, etc.
